# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# Loan CSV file generated from WeatherPy Folder
weather_data = "city_weather.csv"
weather_data1 = pd.read_csv(weather_data)
dropna_weather_ = weather_data1.dropna()
#del dropna_weather["Unnamed: 0"]
#dropna_weather_data.head(20)
weather_data1

,Unnamed: 0,City,Cloudiness,Humidity,Lat,Lng,Max Temp,Windspeed
0,0,Nichinan,75,73,31.6000,131.3667,71.60,8.05
1,1,Cherskiy,95,86,68.7500,161.3000,29.34,4.16
2,2,Pangnirtung,98,95,66.1451,-65.7125,26.69,1.03
3,3,São José da Coroa Grande,28,88,-8.8978,-35.1478,75.38,9.08
4,4,Longyearbyen,90,86,78.2186,15.6401,30.20,3.44
...,...,...,...,...,...,...,...,...
552,552,Prainha,97,97,-1.8000,-53.4800,72.91,2.53
553,553,Athabasca,6,43,54.7169,-113.2854,56.98,1.14
554,554,Burns Lake,46,28,54.2331,-125.7533,57.20,4.61
555,555,Omsukchan,100,93,62.5333,155.8000,30.74,4.23


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [35]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data1[["Lat", "Lng"]]

humidity = weather_data1["Humidity"].astype(float)

In [36]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:

# Filter vacation with zero cloudiness
no_cloud = weather_data1[weather_data1["Cloudiness"] == 0]
# Filter vacation with max temp above 70 degrees F
above_70_degrees = no_cloud[no_cloud["Max Temp"] > 70]
# Filter vacation with max temp below 80 degrees F
below_80_degrees = above_70_degrees[above_70_degrees["Max Temp"] < 80]
# Filter vacation with wind speed below 10 mph
slow_wind = below_80_degrees[below_80_degrees["Windspeed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = slow_wind[slow_wind["Humidity"] < 60]
# Set Index
perfect_vacation = perfect_vacation.reset_index()
del perfect_vacation["index"]
perfect_vacation

,Unnamed: 0,City,Cloudiness,Humidity,Lat,Lng,Max Temp,Windspeed
0,423,Awjilah,0,35,29.1081,21.2869,73.02,7.02
1,455,Najrān,0,21,17.4924,44.1277,73.40,1.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
vaca_locations = perfect_vacation[["Lat", "Lng"]]

vaca_humidity = perfect_vacation["Humidity"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig


Figure(layout=FigureLayout(height='420px'))

In [42]:
# Hotel v
hotels = []

# Loop
for city in range(len(perfect_vacation["City"])):

    lat = perfect_vacation.loc[city]["Lat"]
    lng = perfect_vacation.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
perfect_vacation["Nearest Hotel"] = hotels
perfect_vacation

,Unnamed: 0,City,Cloudiness,Humidity,Lat,Lng,Max Temp,Windspeed,Nearest Hotel
0,423,Awjilah,0,35,29.1081,21.2869,73.02,7.02,Nearest hotel not found
1,455,Najrān,0,21,17.4924,44.1277,73.40,1.57,محمد هادي ال فطيح


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_vacation.iterrows()]
locations = perfect_vacation[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [46]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display Map
vaca_fig

IndexError: list index out of range